In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "0"
os.environ["XLA_FLAGS"] = "--xla_force_host_platform_device_count=32"

import numpy as np
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt

jax.config.update("jax_enable_x64", True)
jax.config.update("jax_platforms", 'cpu')

from ImageD11.sinograms.point_by_point import PBPRefine
import transform as mytrans
import index as myidx

%matplotlib widget

In [2]:
os.chdir('../../tests/data/sandbox/refine/FeAu_0p5_tR_nscope/FeAu_0p5_tR_nscope_top_200um/')

In [3]:
refman = PBPRefine.from_h5('FeAu_0p5_tR_nscope_top_200um_refine_manager_Fe.h5')

Loading peaks
Loading input map
Loading output map


In [4]:
os.chdir('../../../../../../anri/sandbox/')

In [5]:
# from ImageD11.sinograms import geometry
# import subprocess
# from ImageD11 import unitcell
# from ImageD11.sinograms.point_by_point import unitcell_to_b, PBPMap

# def run_refine(self, points_step_space=None, npoints=None):
#     """
#     Call ImageD11.sinograms.point_by_point.refine_map to refine
#     If you say use_cluster, will save the refinement object to disk
#     Then submit a job to slurm to refine
#     """
#     # get columns for refine.pbpmap
#     # which contain the ri, rj points
#     # right now it's indexed in step space
#     ri_col, rj_col = geometry.step_to_recon(self.pbpmap.i, self.pbpmap.j, self.mask.shape)

#     if points_step_space is None:
#         # get the list of peak indices masks in reconstruction space
#         points_recon_space = np.array(np.nonzero(self.mask)).T
#         if npoints is not None:
#             points_recon_space = points_recon_space[:npoints]
#             # print(points_recon_space)
#     else:
#         # convert input points to reconstruction space, then pass to the function
#         points_recon_space = [geometry.step_to_recon(si, sj, self.mask.shape) for (si, sj) in points_step_space]

#     # columnfile by [3, 3, (ri, rj)]
#     all_pbpmap_ubis = self.pbpmap.ubi

#     pars = self.icolf.parameters.get_parameters()

#     dummy_var = np.eye(3)
#     uc = unitcell.unitcell_from_parameters(self.icolf.parameters)
#     B0 = unitcell_to_b(uc.lattice_parameters, dummy_var)

#     origin_sample = np.zeros((self.icolf.nrows, 3))
#     origin_sample[:, 0] = self.icolf.xpos_refined

#     xyz_lab = np.column_stack((self.icolf.xl,
#                       self.icolf.yl,
#                       self.icolf.zl))

#     final_ubis, final_eps, final_npks = refine_map(all_pbpmap_ubis,
#                                                   self.pbpmap.i, self.pbpmap.j,
#                                                   self.sx_grid, self.sy_grid,
#                                                   self.mask,
#                                                   self.icolf.omega,
#                                                   xyz_lab,
#                                                   origin_sample,
#                                                   self.icolf.dty,
#                                                   self.ystep,
#                                                   self.y0,
#                                                   pars.get('wedge'),
#                                                   pars.get('chi'),
#                                                   pars.get('wavelength'),
#                                                   self.hkl_tol_refine,
#                                                   B0)

#      # now we match how the indexer returns dodgy values
#     # mask nan 3x3 entires to identity matrix
#     final_ubis.at[:, :, jnp.isnan(final_ubis[0, 0, :])].set(jnp.eye(3)[..., None])
#     # replace nans with 0 in the eps
#     final_eps.at[:, :, jnp.isnan(final_eps[0, 0, :])].set(0)
#     # replace nans with 0 in the npks
#     final_npks.at[jnp.isnan(final_npks)].set(0)
#     # # replace nans with 0 in the nuniq
#     # final_nuniq[np.isnan(final_nuniq)] = 0

#     # make a PBPMap to contain the output, then save to disk
#     output_map = PBPMap(new=True)
#     output_map.nrows = final_ubis.shape[0]
#     final_ubis_cols = final_ubis.reshape(9, final_ubis.shape[0])
#     final_eps_cols = final_eps.reshape(9, final_eps.shape[0])
#     ubi00, ubi01, ubi02, ubi10, ubi11, ubi12, ubi20, ubi21, ubi22 = final_ubis_cols
#     eps00, eps01, eps02, eps10, eps11, eps12, eps20, eps21, eps22 = final_eps_cols

#     print(final_npks.shape)

#     # add the columns
#     output_map.addcolumn(self.pbpmap.i, 'i')
#     output_map.addcolumn(self.pbpmap.j, 'j')
#     output_map.addcolumn(final_npks, 'ntotal')
#     # output_map.addcolumn(final_nuniq, 'nuniq')
#     output_map.addcolumn(ubi00, 'ubi00')
#     output_map.addcolumn(ubi01, 'ubi01')
#     output_map.addcolumn(ubi02, 'ubi02')
#     output_map.addcolumn(ubi10, 'ubi10')
#     output_map.addcolumn(ubi11, 'ubi11')
#     output_map.addcolumn(ubi12, 'ubi12')
#     output_map.addcolumn(ubi20, 'ubi20')
#     output_map.addcolumn(ubi21, 'ubi21')
#     output_map.addcolumn(ubi22, 'ubi22')
#     output_map.addcolumn(eps00, 'eps00')
#     output_map.addcolumn(eps01, 'eps01')
#     output_map.addcolumn(eps02, 'eps02')
#     output_map.addcolumn(eps10, 'eps10')
#     output_map.addcolumn(eps11, 'eps11')
#     output_map.addcolumn(eps12, 'eps12')
#     output_map.addcolumn(eps20, 'eps20')
#     output_map.addcolumn(eps21, 'eps21')
#     output_map.addcolumn(eps22, 'eps22')

#     self.refinedmap = output_map

In [ ]:
# @jax.jit
# def strain_fit(ydist, gve, hkl, U, B0, gnoise_std, weight_mask):
#     def remove_outliers(w):
#         mean = jnp.mean(w)
#         std = jnp.std(w)
#         upper = mean + 3.5 * std
#         lower = mean - 3.5 * std
#         # Mask elements outside the allowed range
#         mask = (w >= lower) & (w <= upper)
#         return w * mask  # zeros out outliers

#     gve0 = jnp.einsum('ij,nj->ni', U.dot(B0), hkl)
#     gTg0 = jnp.sum(gve * gve0, axis=1)
#     gTg = jnp.sum(gve * gve, axis=1)
#     directional_strain = (gTg0 / gTg) - 1

#     kappa = gve / jnp.linalg.norm(gve, axis=1)[:, None]
#     kx = kappa[:, 0]
#     ky = kappa[:, 1]
#     kz = kappa[:, 2]

#     M = jnp.column_stack((kx * kx, ky * ky, kz * kz, 2 * kx * ky, 2 * kx * kz, 2 * ky * kz))

#     w = (1. / (ydist + 1)).reshape(gve.shape[0], 1)
#     # The noise in the directional strain now propagates according to the linear transform
#     a = jnp.sum(gve0 * (gnoise_std ** 2) * gve0, axis=1)
#     strain_noise_std = jnp.sqrt(divide_where(a, gTg ** 2, out=jnp.ones_like(gTg), wherearr=gTg))
#     print(strain_noise_std)

#     w = w * (1. / strain_noise_std.reshape(w.shape))

#     w = remove_outliers(w)
#     w *= weight_mask.astype(gve.dtype)[:, None]

#     w = w / jnp.max(w)
#     a = w * M
#     b = w.flatten() * directional_strain
#     m, n = a.shape[-2:]
#     rcond = jnp.finfo(b.dtype).eps * max(n, m)
#     eps_vec = jnp.linalg.lstsq(a, b, rcond=rcond)[0].T
#     sxx, syy, szz, sxy, sxz, syz = eps_vec
#     eps_tensor = jnp.array([[sxx, sxy, sxz], [sxy, syy, syz], [sxz, syz, szz]])
#     return eps_tensor


In [ ]:

# @jax.jit
# def refine_pixel(args):
#     ubi_in, ri, rj = args

#     xi0 = sx_grid[ri,rj]
#     yi0 = sy_grid[ri,rj]

#     # mask to location in space
#     sino_mask, ydist = get_voxel_idx(y0, xi0, yi0, omega, dty, ystep)
    
#     # mask g-vectors to those indexed by this ubi
#     hkl, hkl_int = myidx.gv_ubi_to_hkl_int(ubi_in, gve_all)
#     dhkl = hkl - hkl_int
#     drlv2 = jnp.sum(dhkl**2, axis=1)
#     # apply sino_mask here
#     score_mask = (drlv2 < hkl_tol**2) & sino_mask

#     # perform least-squares fit on the UBI with mask to zero the weights from pixels we don't care about
#     w, ubifit, residuals, rank, sing_vals = weighted_lstsq_ubi_fit(ydist, gve_all, hkl_int, score_mask)

#     # how many peaks do we index after refinement?
#     score_mask_post_refine = myidx.score_mask(ubifit, gve_all, hkl_tol) & sino_mask
#     score_post_refine = jnp.sum(score_mask_post_refine)

#     ucell = ubi_to_unitcell(ubifit)
#     U = ubi_and_ucell_to_u(ubifit, ucell)
    
#     # fit the strains with the peaks we index after refinement
#     eps_tensor = strain_fit(ydist, gve_all, hkl_int, U, B0, gnoise_std, score_mask_post_refine)

#     return ubifit, eps_tensor, score_post_refine


# @jax.jit
# def per_pixel(ubi_in, si, sj):
#     ri, rj = step_to_recon(si, sj, recon_shape)
#     ri = jnp.rint(ri).astype(int)
#     rj = jnp.rint(rj).astype(int)

#     # Choose if we want to refine this pixel
#     # Also check that the ubi in is not nans
#     refine_here = jnp.logical_and(recon_mask[ri, rj], ubi_in.at[0,0] is not jnp.nan)
    
#     # the below is equivalent to:
#     # if refine_here:
#     #     return refine_pixel((ubi_in, ri, rj))
#     # else:
#     #     return np.eye(3) - 1
#     ubi_out, eps_out, npks_out = jax.lax.cond(refine_here,  # Condition
#                                    refine_pixel,  # True branch
#                                    lambda _: (jnp.full((3,3),jnp.nan),jnp.full((3,3),jnp.nan), 1),  # False branch
#                                    operand=(ubi_in, ri, rj))

#     return ubi_out, eps_out, npks_out


In [ ]:
def prepare_input_data(self):
    from ImageD11.sinograms import geometry
    import subprocess
    from ImageD11 import unitcell
    from ImageD11.sinograms.point_by_point import unitcell_to_b, PBPMap
    
    # columnfile by [3, 3, (ri, rj)]
    all_pbpmap_ubis = self.pbpmap.ubi
    
    pars = self.icolf.parameters.get_parameters()
    
    dummy_var = np.eye(3)
    uc = unitcell.unitcell_from_parameters(self.icolf.parameters)
    B0 = unitcell_to_b(uc.lattice_parameters, dummy_var)
    
    origin_sample = np.zeros((self.icolf.nrows, 3))
    origin_sample[:, 0] = self.icolf.xpos_refined
    
    xyz_lab = np.column_stack((self.icolf.xl,
                      self.icolf.yl,
                      self.icolf.zl))
    
    gve_all = mytrans.xyz_lab_to_g(xyz_lab, self.icolf.omega, origin_sample, pars.get('wedge'), pars.get('chi'), pars.get('wavelength'))
    
    dty = self.icolf.dty
    ystep = self.ystep
    y0 = self.y0
    hkl_tol = self.hkl_tol_refine
    si = self.pbpmap.i
    sj = self.pbpmap.j
    
    recon_mask = self.mask
    recon_shape = recon_mask.shape
    ri, rj = step_to_recon(si, sj, recon_shape)
    ri = np.round(ri).astype(int)
    rj = np.round(rj).astype(int)
    sx_grid = self.sx_grid
    sy_grid = self.sy_grid
    omega = self.icolf.omega
    gnoise_std = 1e-4

    ubit = jnp.transpose(all_pbpmap_ubis, (2, 0, 1)).block_until_ready()

    return ubit, ri, rj, recon_mask, sx_grid, sy_grid, gve_all, omega, dty, B0, ystep, y0, self.hkl_tol_refine, gnoise_std

In [ ]:

def step_to_recon(si, sj, recon_shape):
    """Converts step space (si, sj) to reconstruction space (ri, rj)"""
    ri = si + (recon_shape[0] // 2)
    rj = sj + (recon_shape[1] // 2)
    return ri, rj


def get_voxel_idx(y0, xi0, yi0, omega, dty, ystep):
    """
    get peaks at xi0, yi0
    basically just geometry.dty_values_grain_in_beam_sincos
    """
    
    ydist = jnp.abs(y0 - xi0 * jnp.sin(omega) - yi0 * jnp.cos(omega) - dty)
    mask = ydist <= ystep

    return mask, ydist


def weighted_lstsq_ubi_fit(ydist, gve, hkl, weight_mask):
    # run the weighted fit
    # a.T @ gve = h =>  gve.T @ a = h.T => a = np.linalg.pinv(gve.T) @ h.T, same for b and c
    w = (1. / (ydist + 1)).reshape(gve.shape[0], 1)
    w *= weight_mask.astype(gve.dtype)[:, None]
    a = w * gve
    b = w * hkl
    m, n = a.shape[-2:]
    rcond = jnp.finfo(b.dtype).eps * max(n, m)
    ubifitT, residuals, rank, sing_vals = jnp.linalg.lstsq(a, b, rcond=rcond)
    ubifit = ubifitT.T

    return w, ubifit, residuals, rank, sing_vals


def divide_where(arr1, arr2, out, wherearr):
    """
    Do arr1/arr2.
    In locations where wherearr == 0, return out instead
    """
    div = jnp.divide(arr1, arr2)
    return jnp.where(wherearr != 0, div, out)


def strain_fit(ydist, gve, M, hkl, U, B0, gnoise_std, weight_mask, eps_reg=1e-12):
    """Optimized strain fit.
    
    Shapes (assumed fixed across calls):
    - gve: (N, 3)
    - hkl: (N, 3)
    - ydist: (N,) or (N,1)
    - weight_mask: (N,) or (N,1)
    
    
    Returns:
    3x3 symmetric strain tensor (dtype matches inputs)
    """
    
    
    # Helper: robust division for small denominators
    def safe_div(num, den, default=0.0, eps=1e-12):
        den_pos = jnp.where(jnp.abs(den) > eps, den, jnp.nan)
        return jnp.where(jnp.isfinite(den_pos), num / den_pos, default)
    
    def remove_outliers(vec):
        mean = jnp.mean(vec)
        std = jnp.std(vec)
        lo = mean - 3.5 * std
        hi = mean + 3.5 * std
        return jnp.where((vec >= lo) & (vec <= hi), vec, 0.0)
    
    # gve0: projection of hkl through U.dot(B0)
    UB = jnp.dot(U, B0)
    # einsum is fine for this contraction; keeps shapes clear
    gve0 = jnp.einsum('ij,nj->ni', UB, hkl)
    
    gTg0 = jnp.sum(gve * gve0, axis=1)
    gTg = jnp.sum(gve * gve, axis=1)
    
    directional_strain = gTg0/gTg - 1
    
    # Base weights from geometry
    w = 1.0 / (ydist.reshape(-1) + 1.0)
    
    # Propagated noise in directional strain
    a_noise = jnp.sum(gve0 * (gnoise_std ** 2) * gve0, axis=1)
    strain_noise_std = jnp.sqrt(safe_div(a_noise, (gTg ** 2), default=1.0))
    
    # Combine weights and apply mask
    w = w / jnp.maximum(strain_noise_std, 1e-12)
    w = jnp.where(jnp.isfinite(w), w, 0.0)
    
    # Remove outliers more than 3.5 std away
    w = remove_outliers(w)

    # Apply provided mask (ensure same shape)
    wm = weight_mask.reshape(-1)
    w = w * wm.astype(gve.dtype)

    # normalize
    wmax = jnp.maximum(jnp.max(w), 1e-12)
    w = w / wmax
    
    # Weighted design and RHS
    A = (w[:, None] * M) # shape (N, 6)
    b = w * directional_strain # shape (N,) 
    
    # Solve normal equations with tiny Tikhonov regularization for stability
    ATA = A.T @ A # shape (6,6)
    ATb = A.T @ b # shape (6,)

    # scale-robust regularization: scale by trace of ATA
    trace = jnp.trace(ATA)
    scale = jnp.maximum(trace / 6.0, 1.0)
    reg = eps_reg * scale
    ATA_reg = ATA + reg * jnp.eye(6, dtype=ATA.dtype)
    
    eps_vec = jnp.linalg.solve(ATA_reg, ATb) # shape (6,)
    
    # Map to symmetric strain tensor
    sxx, syy, szz, sxy, sxz, syz = eps_vec
    eps_tensor = jnp.array([
        [sxx, sxy, sxz],
        [sxy, syy, syz],
        [sxz, syz, szz],
    ], dtype=eps_vec.dtype)
    
    # scale-robust regularization: scale by trace of ATA
    return eps_tensor
    

def ubi_to_unitcell(ubi):
    mt = jnp.dot(ubi, ubi.T)
    G = mt
    a, b, c = jnp.sqrt(jnp.diag(G))
    al = jnp.degrees(jnp.arccos(G[1, 2] / b / c))
    be = jnp.degrees(jnp.arccos(G[0, 2] / a / c))
    ga = jnp.degrees(jnp.arccos(G[0, 1] / a / b))
    return jnp.array([a, b, c, al, be, ga])


def ubi_and_ucell_to_u(ubi, ucell):
    # compute B
    a, b, c = ucell[:3]
    ralpha, rbeta, rgamma = jnp.radians(ucell[3:])  # radians
    ca = jnp.cos(ralpha)
    cb = jnp.cos(rbeta)
    cg = jnp.cos(rgamma)
    g = jnp.full((3, 3), jnp.nan, jnp.float64)
    g = g.at[0, 0].set(a * a)
    g = g.at[0, 1].set(a * b * cg)
    g = g.at[0, 2].set(a * c * cb)
    g = g.at[1, 0].set(a * b * cg)
    g = g.at[1, 1].set(b * b)
    g = g.at[1, 2].set(b * c * ca)
    g = g.at[2, 0].set(a * c * cb)
    g = g.at[2, 1].set(b * c * ca)
    g = g.at[2, 2].set(c * c)
    gi = jnp.linalg.inv(g)
    astar, bstar, cstar = jnp.sqrt(jnp.diag(gi))
    betas = jnp.degrees(jnp.arccos(gi[0, 2] / astar / cstar))
    gammas = jnp.degrees(jnp.arccos(gi[0, 1] / astar / bstar))

    B = jnp.zeros((3, 3))

    B = B.at[0, 0].set(astar)
    B = B.at[0, 1].set(bstar * jnp.cos(jnp.radians(gammas)))
    B = B.at[0, 2].set(cstar * jnp.cos(jnp.radians(betas)))
    B = B.at[1, 0].set(0.0)
    B = B.at[1, 1].set(bstar * jnp.sin(jnp.radians(gammas)))
    B = B.at[1, 2].set(-cstar * jnp.sin(jnp.radians(betas)) * ca)
    B = B.at[2, 0].set(0.0)
    B = B.at[2, 1].set(0.0)
    B = B.at[2, 2].set(1.0 / c)

    u = jnp.dot(B, ubi).T
    return u

In [ ]:
800*12/60/60

In [ ]:
# hkl_calc

for voxel in voxels:
    # get g_calc from ubi and hkl
    # g_calcs give omega_calc
    # omega_calc gives dty_calc
    # (omega_calc, dty_calc) is a sinogram frame
    # find the best peak in that frame
    # 
    # for each hkl_calc, get the best peak from gcalc and gves
    # so for each grain we have an (npeaks_calc, ) array
    # 

In [ ]:
# you have UBI in a pixel
# you have (dty, omega) for that pixel
# you have theoretical hkls0
# forward project gve_calc
# this gives you theoretical angles for each peak
# sort cf_2d by omega
# for the grain, fill in the peaks to grains problem
# 

In [ ]:
from functools import partial


def refine_pixel_single(ubi_in, ri_px, rj_px, 
                        rm, sx_grid, sy_grid, 
                        gve, M, omega, dty,
                        B0,
                        ystep, y0,
                        hkl_tol,
                        gnoise_std):
    refine_here = jnp.logical_and(rm[ri_px, rj_px],
                                  ~jnp.isnan(ubi_in[0,0]))

    def do_refine(ubi):
        xi0 = sx_grid[ri_px, rj_px]
        yi0 = sy_grid[ri_px, rj_px]

         # npks mask
        sino_mask, ydist = get_voxel_idx(y0, xi0, yi0, omega, dty, ystep)

        hkl, hkl_int = myidx.gv_ubi_to_hkl_int(ubi, gve)
        dhkl = hkl - hkl_int
        drlv2 = jnp.sum(dhkl**2, axis=1)
        
        score_mask = (drlv2 < hkl_tol**2) & sino_mask

        # there is an array of theoretical hkls based on how many rings hit your detector
        # per grain
        # h, k, l, etasign
        # each field in this array should be a peak id or -1
        # could also have a weight
        w, ubifit, residuals, rank, sing_vals = weighted_lstsq_ubi_fit(ydist, gve, hkl_int, score_mask)

        score_mask_post_refine = myidx.score_mask(ubifit, gve, hkl_tol) & sino_mask
        score_post_refine = jnp.sum(score_mask_post_refine)

        ucell = ubi_to_unitcell(ubifit)
        U = ubi_and_ucell_to_u(ubifit, ucell)

        eps_tensor = strain_fit(ydist, gve, M, hkl_int, U, B0, gnoise_std, score_mask_post_refine)
        # eps_tensor = jnp.eye(3)

        return ubifit, eps_tensor, score_post_refine

    def skip_refine(u):
        return (jnp.full((3,3), jnp.nan), jnp.full((3,3), jnp.nan), 1)

    return jax.lax.cond(refine_here, do_refine, skip_refine, operand=ubi_in)

In [ ]:
ubit, ri, rj, recon_mask, sx_grid, sy_grid, gve_all, omega, dty, B0, ystep, y0, tol, gnoise = prepare_input_data(refman)

In [ ]:
type(sx_grid)

In [ ]:
# unit direction kappa (safe norm)
gve_norm = jnp.linalg.norm(gve_all, axis=1, keepdims=True)
# gve_norm = jnp.maximum(gve_norm, 1e-12)
kappa = gve_all / gve_norm
kx = kappa[:, 0]
ky = kappa[:, 1]
kz = kappa[:, 2]

# Build design matrix M explicitly (N x 6)
M = jnp.stack([
    kx * kx,
    ky * ky,
    kz * kz,
    2.0 * kx * ky,
    2.0 * kx * kz,
    2.0 * ky * kz,
], axis=1)

stat_fun = partial(refine_pixel_single,
                   rm=jnp.array(recon_mask),
                   sx_grid=jnp.array(sx_grid),
                   sy_grid=jnp.array(sy_grid),
                   gve=gve_all,
                   M=M,
                   omega=omega,
                   dty=dty,
                   B0=B0,
                   ystep=ystep,
                   y0=y0,
                   hkl_tol=tol,
                   gnoise_std=gnoise)

stat_fun_jit = jax.jit(stat_fun)

In [ ]:
i = 50000
ubi_in = ubit[i]
ri_in = ri[i]
rj_in = rj[i]

print(ubi_in)

res = stat_fun_jit(ubi_in, jnp.array(ri_in), jnp.array(rj_in))
res = tuple(r.block_until_ready() for r in res)

print(res)

In [ ]:
refine_pixel_batch = jax.pmap(stat_fun_jit)

In [ ]:
# refine_pixel_patch_jit = jax.jit(refine_pixel_batch)

In [ ]:
from tqdm import tqdm

ubis_out_list = []
eps_out_list = []
scores_out_list = []

i = 0
chunk_size = 32
N = ubit.shape[0]

for i in tqdm(range(0, N, chunk_size)):
    end = min(i + chunk_size, N)
    # print(i, end)
    ubi_chunk = ubit[i:end]
    ri_chunk = ri[i:end]
    rj_chunk = rj[i:end]

    # Evaluate this chunk with vmap + jit
    res = refine_pixel_batch(ubi_chunk, ri_chunk, rj_chunk)
    res = tuple(r.block_until_ready() for r in res)
    
    # Append in order
    ubis_out_list.append(res[0])
    eps_out_list.append(res[1])
    scores_out_list.append(res[2])

In [ ]:
from tqdm import tqdm

ubis_out_list = []
eps_out_list = []
scores_out_list = []

for i in tqdm(range(ubit.shape[0])):
    res = stat_fun_jit(ubit[i], ri[i], rj[i])
    res = tuple(r.block_until_ready() for r in res)
    
    # Append in order
    ubis_out_list.append(res[0])
    eps_out_list.append(res[1])
    scores_out_list.append(res[2])

In [ ]:
ubis_out_list

In [ ]:
# define it over some chunk of ubis

refine_pixel_batch = jax.pmap?

In [ ]:


chunk_size = 10
N = ubit.shape[0]

end = min(i + chunk_size, N)
print(i, end)
ubi_chunk = ubit[i:end]
ri_chunk = ri[i:end]
rj_chunk = rj[i:end]

ubis_chunk, eps_chunk, scores_chunk = refine_pixel_batch(ubi_chunk, ri_chunk, rj_chunk)
print(ubis_chunk)

In [ ]:
%%time 
ubis_chunk, eps_chunk, scores_chunk = refine_pixel_batch(ubi_chunk, ri_chunk, rj_chunk)
print(ubis_chunk)
print(eps_chunk)
print(scores_chunk)

In [ ]:
from tqdm import tqdm

ubis_out_list = []
eps_out_list = []
scores_out_list = []

for i in tqdm(range(0, N, chunk_size)):
    end = min(i + chunk_size, N)
    # print(i, end)
    ubi_chunk = ubit[i:end]
    ri_chunk = ri[i:end]
    rj_chunk = rj[i:end]

    # Evaluate this chunk with vmap + jit
    ubis_chunk, eps_chunk, scores_chunk = refine_pixel_batch(ubi_chunk, ri_chunk, rj_chunk)

    # Append in order
    ubis_out_list.append(jax.device_get(ubis_chunk))
    eps_out_list.append(eps_chunk)
    scores_out_list.append(scores_chunk)

In [ ]:
from tqdm import tqdm

for i in tqdm(range(ubit.shape[0])):
    res = stat_fun_jit(ubi_in, jnp.array(ri_in), jnp.array(rj_in))
    res = tuple(r.block_until_ready() for r in res)
    if i % 10000 == 0:
        print(i)
        print(res)

In [ ]:
refine_pixel_batch = jax.vmap(stat_fun_jit)

# refine_pixel_batch_jit = jax.jit(refine_pixel_batch)

In [ ]:
ubis_out_list = []
eps_out_list = []
scores_out_list = []

chunk_size = 1000
N = ubit.shape[0]

for i in range(0, N, chunk_size):
    end = min(i + chunk_size, N)
    print(i, end)
    ubi_chunk = ubit[i:end]
    ri_chunk = ri[i:end]
    rj_chunk = rj[i:end]

    # Evaluate this chunk with vmap + jit
    ubis_chunk, eps_chunk, scores_chunk = refine_pixel_batch(ubi_chunk, ri_chunk, rj_chunk)

    # Append in order
    ubis_out_list.append(jax.device_get(ubis_chunk))
    eps_out_list.append(eps_chunk)
    scores_out_list.append(scores_chunk)


In [ ]:
# --- batch version ---
# ubis_in shape: (N,3,3), ri_array and rj_array shape: (N,)
# Pass gve_all as closure argument
refine_pixel_batch = jax.vmap(refine_pixel_single, in_axes=
                              (0,0,0,
                               None,None,None,
                               None,None,None,
                               None,
                               None,None,
                               None,
                               None,))

# --- optionally JIT the whole batch ---
refine_pixel_batch_jit = jax.jit(refine_pixel_batch)

# ubis_out, eps_out, scores_out = refine_pixel_batch_jit(
#     ubit, ri, rj,
#     recon_mask, sx_grid, sy_grid,
#     gve_all, omega, dty,
#     B0,
#     ystep, y0,
#     tol,
#     gnoise
# )

In [ ]:
refine_pixel_batch_jit(
        ubit[50000:50100], ri[50000:50100], rj[50000:50100],
        recon_mask, sx_grid, sy_grid,
        gve_all, omega, dty,
        B0,
        ystep, y0,
        tol,
        gnoise
    )

In [ ]:
ubis_out_list = []
eps_out_list = []
scores_out_list = []

from tqdm import tqdm

for i in tqdm(range(ubit.shape[0])):
    i = i + 50000
    ubis_out_list.append(refine_pixel_single(ubit[i], ri[i], rj[i],
                    recon_mask, sx_grid, sy_grid,
                    gve_all, omega, dty,
                    B0,
                    ystep, y0,
                    tol, gnoise))

In [ ]:
ubis_out_list

In [ ]:
ubis_out_list = []
eps_out_list = []
scores_out_list = []

chunk_size = 1000
N = ubit.shape[0]

for i in range(0, N, chunk_size):
    end = min(i + chunk_size, N)
    print(i, end)
    ubi_chunk = ubit[i:end]
    ri_chunk = ri[i:end]
    rj_chunk = rj[i:end]

    # Evaluate this chunk with vmap + jit
    ubis_chunk, eps_chunk, scores_chunk = refine_pixel_batch_jit(
        ubi_chunk, ri_chunk, rj_chunk,
        recon_mask, sx_grid, sy_grid,
        gve_all, omega, dty,
        B0,
        ystep, y0,
        tol,
        gnoise
    )

    # Append in order
    ubis_out_list.append(jax.device_get(ubis_chunk))
    eps_out_list.append(eps_chunk)
    scores_out_list.append(scores_chunk)


In [ ]:
ubis_out[50000]

In [ ]:
chunk_size = 10000
N = ubis_out.shape[0]

ubis_out_list = []
eps_out_list = []
scores_out_list = []

for i in range(0, N, chunk_size):
    end = min(i + chunk_size, N)
    ubi_chunk = ubis_out[i:end]
    eps_chunk = eps_out[i:end]
    scores_chunk = scores_out[i:end]
    ubis_out_list.append(ubi_chunk)
    eps_out_list.append(eps_chunk)
    scores_out_list.append(scores_chunk)

In [ ]:
ubis_out_list[5][0]

In [ ]:
ubis_out = jnp.concatenate(ubis_out_list, axis=0)
eps_out = jnp.concatenate(eps_out_list, axis=0)
scores_out = jnp.concatenate(scores_out_list, axis=0)

In [ ]:
ubis_out

In [ ]:
ubit[50000]

In [ ]:
ubis_out[50000]